In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisosONEHOT.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantesONEHOT.csv")
avisos['nombre_area']=avisos['nombre_area'].astype('category')

avisos_no_one_hot= pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
avisos_no_one_hot=avisos_no_one_hot[['idaviso','tipo_de_trabajo','nivel_laboral']];
avisos_no_one_hot['tipo_de_trabajo']=avisos_no_one_hot['tipo_de_trabajo'].astype('category')
avisos_no_one_hot['nivel_laboral']=avisos_no_one_hot['nivel_laboral'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16502001


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,avisos_no_one_hot,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo_Temporario,tipo_de_trabajo_Voluntario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,cantidad
1468819,1112349013,0zPW3jr,0,27.0,True,False,False,False,False,False,...,False,False,False,False,True,False,False,Full-time,Junior,NaN
3046890,1112423974,EzZXGBb,1,29.0,True,False,False,False,False,False,...,False,False,False,False,True,False,False,Full-time,Junior,NaN
13907608,1112374140,6rQJEpM,0,24.0,True,False,False,False,False,False,...,False,False,False,False,True,False,False,Full-time,Junior,NaN
339667,1112318878,DrlAl0A,0,46.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,NaN
14512520,1112368542,96MkKVp,0,23.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,NaN
8054554,1112262236,W9Ozmoj,0,22.0,True,False,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,NaN
3370099,1112388054,5mrOb1k,0,43.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,NaN
3566386,1112257464,4rPdYBP,0,33.0,True,False,False,False,False,False,...,False,False,False,True,False,False,False,Full-time,Jefe / Supervisor / Responsable,NaN
1566708,1112303651,5mPJXYz,0,22.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,NaN
14307949,1112371069,X95VEaO,0,42.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,NaN


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo_Temporario,tipo_de_trabajo_Voluntario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,cantidad
12070113,1112383482,MVPNLPK,1,33.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000
11279909,1112269767,2zPOV3a,1,21.0,True,False,False,False,False,False,...,False,False,False,False,True,False,False,Full-time,Junior,0.000000
13241488,1112222231,xkPL3Vb,0,28.0,True,False,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000
5879703,1112224495,X9PYMWK,0,21.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000
2308460,1112262494,akxONmm,1,25.0,True,False,False,False,False,False,...,False,False,False,False,False,False,True,Part-time,Senior / Semi-Senior,0.000000
5776415,1112312616,96zbOO5,1,20.0,True,False,False,False,False,False,...,False,False,False,False,True,False,False,Full-time,Junior,0.000000
11931575,1112369871,96zGAM1,0,24.0,True,False,False,False,False,False,...,False,False,False,False,True,False,False,Full-time,Junior,0.000000
15916631,1112209124,EmK4q8,0,39.0,False,True,False,False,False,False,...,False,False,False,False,False,True,False,Full-time,Otro,0.000000
10455441,1112418721,mz505wQ,1,36.0,True,False,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,0.100000
2422786,1112347174,PmJ0N1Y,1,21.0,True,False,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000


In [15]:
postulaciones_por_area=postulaciones.copy()
postulaciones_por_area=postulaciones_por_area.groupby(by=['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_area=postulaciones_por_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_por_area.head()

,idpostulante,nombre_area,postulaciones_misma_area
0,0z5Dmrd,Abastecimiento,1
1,0z5Dmrd,Administración,1
2,0z5Dmrd,Administración de Personal,1
3,0z5Dmrd,Almacén / Depósito / Expedición,1
4,0z5Dmrd,Atención al Cliente,1


In [16]:
postulaciones_por_nivel=postulaciones.copy()
postulaciones_por_nivel=postulaciones_por_nivel.groupby(by=['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_nivel=postulaciones_por_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})
postulaciones_por_nivel.head()

,idpostulante,nivel_laboral,postulaciones_mismo_nivel
0,0z5Dmrd,Gerencia / Alta Gerencia / Dirección,1
1,0z5Dmrd,Junior,3
2,0z5Dmrd,Otro,1
3,0z5Dmrd,Senior / Semi-Senior,17
4,0z5JW1r,Gerencia / Alta Gerencia / Dirección,2


In [17]:
postulaciones_por_tipo=postulaciones.copy()
postulaciones_por_tipo=postulaciones_por_tipo.groupby(by=['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_tipo=postulaciones_por_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_por_tipo.head()

,idpostulante,tipo_de_trabajo,postulaciones_mismo_tipo
0,0z5Dmrd,Full-time,20
1,0z5Dmrd,Part-time,1
2,0z5Dmrd,Teletrabajo,1
3,0z5JW1r,Full-time,28
4,0z5JW1r,Part-time,1


In [18]:
postulaciones=pd.merge(postulaciones,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
13917461,1112488217,2zmqkjw,0,48.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,0,16,14
4473941,1112318974,LNK9ZPr,0,26.0,True,False,False,False,False,False,...,False,True,False,False,Full-time,Junior,0.000000,3,18,2
2793982,1002471470,EzO8GLo,1,22.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.009901,13,55,40
9689298,1112305789,ewmDd2,0,64.0,False,True,False,False,False,False,...,False,True,False,False,Full-time,Junior,0.000000,1,17,5
11902243,1112412413,zvxVMKk,1,32.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.002478,129,173,129
7101496,1112389250,jkVjvWY,0,24.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,0,18,16
15612067,1111449788,X95Da2K,0,32.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,0,27,23
12118756,1112226248,Bm5zdlZ,0,22.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,0,24,18
734337,1112273320,mzbLEZd,1,31.0,True,False,False,False,False,False,...,False,True,False,False,Full-time,Junior,0.000000,8,39,8
1534371,1112232520,96XQzY9,0,21.0,True,False,False,False,False,False,...,False,False,False,False,Full-time,Gerencia / Alta Gerencia / Dirección,0.000000,0,19,1


In [19]:
areas=postulaciones['nombre_area'].value_counts()
areas.values.mean()

85564.77659574468

In [20]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0

Train:  15281869 Test:  804309


In [21]:
c=0
gc.collect()

96

In [22]:
areas

Ventas                                       2095013
Administración                               1250431
Comercial                                    1131371
Producción                                   1108817
Atención al Cliente                           652590
Recepcionista                                 424366
Contabilidad                                  420713
Call Center                                   405704
Programación                                  390003
Tecnologia / Sistemas                         329687
Almacén / Depósito / Expedición               263991
Telemarketing                                 262890
Mantenimiento                                 260435
Recursos Humanos                              256353
Tesorería                                     251430
Logística                                     242594
Gastronomia                                   242104
Mantenimiento y Limpieza                      232992
Soporte Técnico                               

In [25]:
#features.remove('nombre_area')
features.remove('tipo_de_trabajo')
features.remove('nivel_laboral')
for i in range(0,len(areas)):
    if(areas.values[i])>100000:
        c=c=MLPClassifier(activation='tanh', alpha=1e-05, batch_size='auto',beta_1=0.9, beta_2=0.9,epsilon=1e-08, hidden_layer_sizes=(len(features), 4), learning_rate='constant',learning_rate_init=0.001, max_iter=200,nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,solver='adam', tol=0.0001, validation_fraction=0.1, verbose=True,warm_start=False)
        dt=c.fit(x_train[x_train['nombre_area']==areas.index[i]][features],y_train[x_train['nombre_area']==areas.index[i]])
        joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/NNPorArea/NN "+areas.index[i].replace("/","-")+".pkl")
        score=c.score(x_test[x_test['nombre_area']==areas.index[i]][features],y_test[x_test['nombre_area']==areas.index[i]])*100
        print(areas.index[i]+": "+str(score))
        c=0
        gc.collect()

Iteration 1, loss = 0.31860107
Iteration 2, loss = 0.28752008
Iteration 3, loss = 0.28017515
Iteration 4, loss = 0.27613948
Iteration 5, loss = 0.27386363
Iteration 6, loss = 0.27182313
Iteration 7, loss = 0.27066074
Iteration 8, loss = 0.26945442
Iteration 9, loss = 0.26873280
Iteration 10, loss = 0.26780532
Iteration 11, loss = 0.26761278
Iteration 12, loss = 0.26692278
Iteration 13, loss = 0.26637287
Iteration 14, loss = 0.26618073
Iteration 15, loss = 0.26574044
Iteration 16, loss = 0.26540869
Iteration 17, loss = 0.26511565
Iteration 18, loss = 0.26500486
Iteration 19, loss = 0.26491308
Iteration 20, loss = 0.26472283
Iteration 21, loss = 0.26481303
Iteration 22, loss = 0.26449983
Iteration 23, loss = 0.26424441
Iteration 24, loss = 0.26418482
Iteration 25, loss = 0.26421205
Iteration 26, loss = 0.26396498
Iteration 27, loss = 0.26385170
Iteration 28, loss = 0.26429641
Iteration 29, loss = 0.26433521
Iteration 30, loss = 0.26392338
Training loss did not improve more than tol=0.000

In [26]:
c=0
gc.collect()

0

In [27]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos_no_one_hot,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion=pd.merge(prediccion,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area']
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel']
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo']
prediccion.sample(10)

100000
100000
100000
100000


,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
35301,66402,1112459528,6rPVkel,25.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.030303,NaN,15.0,10.0
78404,39326,1112417420,1pGMwb,37.0,True,False,False,False,False,False,...,True,False,False,False,Full-time,Jefe / Supervisor / Responsable,0.000000,22.0,43.0,2.0
78853,26244,1112338811,EeVpr9,39.0,True,False,False,False,False,False,...,False,True,False,False,Full-time,Junior,0.000000,NaN,38.0,3.0
33580,8104,1111753681,VNEM31N,39.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,NaN,17.0,17.0
42929,34996,1112409703,Z1x32V,46.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,NaN,28.0,23.0
92651,91700,1112472293,ZjY3Q8,39.0,True,False,False,False,False,False,...,False,True,False,False,Full-time,Junior,0.000000,NaN,19.0,3.0
94997,94376,1112484065,6ad6D3,47.0,False,True,False,False,False,False,...,True,False,False,False,Full-time,Jefe / Supervisor / Responsable,0.000000,1.0,15.0,NaN
41112,52349,1112443153,EDwNkz,49.0,True,False,False,False,False,False,...,False,True,False,False,Full-time,Junior,0.000000,NaN,15.0,2.0
70858,19097,1112222393,jkV9DxY,47.0,False,True,False,False,False,False,...,False,False,False,False,Full-time,Gerencia / Alta Gerencia / Dirección,0.058824,8.0,28.0,6.0
2614,13206,1112033906,EzpVwE4,34.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.030303,3.0,29.0,17.0


In [28]:
#Elijo una buena prediccion
buena = pd.read_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion35.csv")
prediccion=pd.merge(prediccion, buena,on='id',how='left')

In [32]:
for i in range(0,len(areas)):
    if(areas.values[i])>100000:
        c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/NNPorArea/NN "+areas.index[i].replace("/","-")+".pkl")
        nueva=prediccion[prediccion['nombre_area']==areas.index[i]]
        x=nueva[features].fillna(0)
        y=c.predict_proba(x)
        y2=[]
        for elem in y:
            y2.append(elem[1])
        nueva['sepostulo']=y2
        prediccion=pd.concat([prediccion, nueva],axis=0,ignore_index=True)
        prediccion.drop_duplicates('id',keep='last',inplace=True)
        c=0
        x=0
        y=0
        y2=0
        nueva=0
        gc.collect()
        print(areas.index[i])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


Ventas
Administración
Comercial
Producción
Atención al Cliente
Recepcionista
Contabilidad
Call Center
Programación
Tecnologia / Sistemas
Almacén / Depósito / Expedición
Telemarketing
Mantenimiento
Recursos Humanos
Tesorería
Logística
Gastronomia
Mantenimiento y Limpieza
Soporte Técnico
Marketing
Oficios y Profesiones
Administración de Personal
Compras
Créditos y Cobranzas
Finanzas
Legal
Transporte
Otros
Seguridad
Secretaria
Salud
Selección
Desarrollo de Negocios
Calidad
Impuestos


In [33]:
prediccion.sort_values('id',inplace=True)
prediccion.head()

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,sepostulo
42715,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,False,False,False,Full-time,Jefe / Supervisor / Responsable,0.0,1.0,19.0,1.0,0.003516
42716,1,739260,6v1xdL,30.0,False,True,False,False,False,False,...,False,False,False,Full-time,Jefe / Supervisor / Responsable,0.0,6.0,64.0,NaN,0.155338
42717,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,False,False,False,Full-time,Jefe / Supervisor / Responsable,0.0,4.0,16.0,2.0,0.007382
71648,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,False,True,False,Full-time,Otro,0.0,2.0,21.0,2.0,0.012192
71649,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,False,True,False,Full-time,Otro,0.0,2.0,17.0,3.0,0.002032


In [34]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,nombre_area,tipo_de_trabajo,nivel_laboral,sepostulo
42715,0,Comercial,Full-time,Jefe / Supervisor / Responsable,0.003516
42716,1,Comercial,Full-time,Jefe / Supervisor / Responsable,0.155338
42717,2,Comercial,Full-time,Jefe / Supervisor / Responsable,0.007382
71648,3,Tecnologia / Sistemas,Full-time,Otro,0.012192
71649,4,Tecnologia / Sistemas,Full-time,Otro,0.002032


In [35]:
prediccion.drop(['nombre_area','tipo_de_trabajo','nivel_laboral'],axis=1,inplace=True)

In [37]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion42.csv",index=False)

In [36]:
prediccion.sepostulo.mean()

0.4814961681530466